# Fine-Tuning BERT for Text Classification

# Dataset Preparation
## Load the CSV

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd # Import the pandas library

file_path = '/content/drive/MyDrive/BertDataset/comments.csv'
df = pd.read_csv(file_path)

## check

In [ ]:
print(f"Total rows in CSV: {len(df)}")
print(df.head())

## Preprocess Text

In [ ]:
# Clean the data (remove special symbols and handle NaN)
def clean_text(text):
    if isinstance(text, str):  # Ensure it's a string before processing
        # Remove the special characters
        text = text.replace('?', '').replace('&', '').replace('#', '').replace('$', '').replace('%', '').replace('@', '').replace('*', '').replace('^', '')
        return text.lower()  # Convert to lowercase
    return ''

In [ ]:
# Apply the cleaning function to the text data
df['text'] = df['text'].apply(clean_text)

# Drop rows with NaN values
df = df.dropna(subset=['text', 'label'])

In [ ]:
# Shuffle the dataset
df = df.sample(frac=1).reset_index(drop=True)

# Check if data looks correct now
print(df.head())

##Split Data (70% Train, 30% Test)

In [8]:
# Split the dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

In [9]:
import re

test_text = "هل هذا مثال؟ #اختبار"
cleaned = re.sub(r'[?&#$%@*^]', '', test_text)
print(f"Before: {test_text}\nAfter: {cleaned}")

Before: هل هذا مثال؟ #اختبار
After: هل هذا مثال؟ اختبار


# BERT Model Setup

## Choose Model

In [10]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

## Tokenization

##Determine max_len

In [12]:
from transformers import AutoTokenizer
from datasets import load_dataset

# Load tokenizer and dataset
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
raw_dataset = load_dataset("imdb")

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length")

# Tokenize dataset
tokenized_datasets = raw_dataset.map(tokenize_function, batched=True)

# Set format to torch tensors
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "label"])

train_ds = tokenized_datasets["train"]
eval_ds = tokenized_datasets["test"]

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [13]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc}

# Training Configuration
## Batch Size Selection

In [14]:
# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Train Setup

In [15]:
!pip install transformers --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 51.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.3
    Uninstalling transformers-4.50.3:
      Successfully uninstalled transformers-4.50.3


In [16]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "offline"
os.environ["TRANSFORMERS_NO_WANDB"] = "1"


In [17]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none",  # 👈 disables wandb
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",  # 👈 matches compute_metrics
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  if self.torch_empty_cache_steps is not None:


In [18]:
from transformers import AutoModelForSequenceClassification, Trainer

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    compute_metrics=compute_metrics,
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


#  Evaluation

In [ ]:
# Evaluate the model
predictions, labels, _ = trainer.predict(torch.utils.data.TensorDataset(test_input_ids, test_attention_mask, test_labels))

# Get the predicted labels
predicted_labels = np.argmax(predictions, axis=1)

## Confusion Matrix

In [ ]:
# Plot the confusion matrix
cm = confusion_matrix(test_labels, predicted_labels)
plt.figure(figsize=(6, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
classes = ['Positive', 'Negative', 'Mix']
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.show()

# Test on Custom Sentences

In [ ]:
custom_sentences = [
    "الفيلم كان رائعاً!",                   # Positive
    "الخدمة مقبولة لكن بطيئة",              # Mixed
    "أسوأ تجربة في حياتي"                   # Negative
]

cleaned_sentences = [clean_text(s) for s in custom_sentences]
encoded_inputs = tokenizer(
    cleaned_sentences,
    truncation=True,
    padding='max_length',
    max_length=max_len,
    return_tensors='pt'
).to(device)

with torch.no_grad():
    outputs = model(**encoded_inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)

reverse_label_map = {v: k for k, v in label_map.items()}
for i, sentence in enumerate(custom_sentences):
    print(f"Sentence: {sentence}")
    print(f"Predicted sentiment: {reverse_label_map[predictions[i].item()]}")
    print("-" * 50)

##